```sql
SELECT order_item_id, order_id, product_id, customer_id, date_created, product_net_revenue FROM wpkr_wc_order_product_lookup; -- order product
```

In [45]:
import pandas as pd

df = pd.read_csv("csv/order_meta.csv", sep=";")
display(df.head(10))
print(f"DF: {df.shape[0]} rows x {df.shape[1]} columns")
display(df.info())
print(df.duplicated().sum())
print(df.isna().sum())

,id,order_id,meta_key,meta_value
0,79,1036,is_vat_exempt,no
1,80,1036,_billing_address_index,Marlene mgreallyo@quantcast.com Kota Pontiana...
2,81,1036,_shipping_address_index,Marlene mgreallyo@quantcast.com Kota Pontiana...
3,82,1036,_wc_order_attribution_source_type,typein
4,83,1036,_wc_order_attribution_utm_source,(direct)
5,84,1036,_wc_order_attribution_session_entry,https://datamininguntan.my.id/
6,85,1036,_wc_order_attribution_session_start_time,2025-05-08 22:32:14
7,86,1036,_wc_order_attribution_session_pages,9
8,87,1036,_wc_order_attribution_session_count,1
9,88,1036,_wc_order_attribution_user_agent,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...


DF: 1302 rows x 4 columns
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1302 entries, 0 to 1301
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          1302 non-null   int64 
 1   order_id    1302 non-null   int64 
 2   meta_key    1302 non-null   object
 3   meta_value  1302 non-null   object
dtypes: int64(2), object(2)
memory usage: 40.8+ KB


None

0
id            0
order_id      0
meta_key      0
meta_value    0
dtype: int64


In [42]:
cols = [col for col in df.columns if df[col].nunique() > 0]
for col in cols:
    print(f"Len {col}: {len(df[col].unique().tolist())}") # unique values
    print(f"{col}: {df[col].unique().tolist()}\n")

Len product_id: 100
product_id: [322, 447, 481, 527, 536, 543, 565, 569, 580, 586, 590, 594, 602, 609, 613, 619, 632, 638, 646, 652, 659, 667, 673, 678, 682, 688, 694, 698, 702, 706, 713, 717, 721, 726, 730, 734, 738, 741, 746, 750, 754, 762, 767, 770, 773, 778, 786, 790, 793, 797, 805, 809, 812, 817, 821, 825, 829, 833, 850, 856, 860, 867, 871, 875, 879, 883, 887, 891, 895, 899, 903, 907, 911, 915, 922, 929, 933, 938, 942, 946, 951, 954, 958, 962, 966, 971, 975, 978, 983, 986, 990, 994, 998, 1002, 1006, 1010, 1015, 1019, 1020, 1064]

Len sku: 2
sku: ['woo-single', nan]

Len virtual: 2
virtual: [1, 0]

Len downloadable: 2
downloadable: [1, 0]

Len min_price: 88
min_price: ['19000,0000', '40000,0000', '299000,0000', '20580,0000', '30500,0000', '499900,0000', '62700,0000', '48000,0000', '55000,0000', '52000,0000', '129000,0000', '149000,0000', '137750,0000', '160000,0000', '121000,0000', '185017,0000', '89995,0000', '89940,0000', '794000,0000', '200000,0000', '50000,0000', '89000,0000', 

In [ ]:
import pandas as pd
import plotly.graph_objects as go
import networkx as nx

# Load dan preprocess data
df_sales = pd.read_csv('csv/fact_sales_v2.csv', sep=';')
if df_sales['total_sales'].dtype == object:
    df_sales['total_sales'] = (
        df_sales['total_sales']
        .str.replace('.', '', regex=False)
        .str.replace(',', '.', regex=False)
        .astype(float)
)
dag_sales = df_sales.groupby('product_id', as_index=False)['total_sales'].sum()

df_order_product = pd.read_csv('csv/order_product.csv', sep=';')
df_merge1 = dag_sales.merge(
    df_order_product[['order_item_id', 'product_id']],
    on='product_id', how='left'
)

df_items = pd.read_csv('csv/order_items.csv', sep=';')
df_merge2 = df_merge1.merge(
    df_items[['order_item_id', 'order_item_name']].drop_duplicates('order_item_id'),
    on='order_item_id', how='left'
)

df_final = df_merge2[['product_id', 'order_item_name', 'total_sales']].copy()
df_final.columns = ['id', 'order_item_name', 'total_sales']
df_final = df_final.drop_duplicates(subset=['id'], keep='first')

# Ambil 5 produk teratas
df_top5 = df_final.sort_values('total_sales', ascending=False).head(5)

# Scaling ukuran bubble
sizes = (df_top5['total_sales'] / df_top5['total_sales'].max()) * 100 + 40  

# Buat graph kosong dan layout spring (bubble cloud)
G = nx.Graph()
for i in range(len(df_top5)):
    G.add_node(i)

pos = nx.spring_layout(G, k=0.5, seed=42)  # posisi node dengan spring layout

x_pos = [pos[i][0] for i in range(len(df_top5))]
y_pos = [pos[i][1] for i in range(len(df_top5))]

# Buat figure Plotly
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=x_pos,
    y=y_pos,
    mode='markers+text',
    text=[f"{name}<br><b>{int(total):,}</b>" for name, total in zip(df_top5['order_item_name'], df_top5['total_sales'])],
    textposition='middle center',
    marker=dict(
        size=sizes,
        color=df_top5['total_sales'],
        colorscale='Viridis',
        showscale=True,
        line=dict(width=2, color='DarkSlateGrey'),
        sizemode='diameter',
        opacity=0.7,
    ),
    hoverinfo='text',
    hovertext=[f"{name}<br>Total Sales: {total:,.0f}" for name, total in zip(df_top5['order_item_name'], df_top5['total_sales'])]
))

fig.update_layout(
    title="Bubble Cloud Chart: Top 5 Produk by Total Penjualan",
    xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
    yaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
    plot_bgcolor='white',
    height=600,
    width=800,
    margin=dict(l=20, r=20, t=60, b=20)
)

# Simpan ke PNG 
output_path = 'bubble_cloud_chart.png'
fig.write_image(output_path, scale=2)

print(f"Chart berhasil disimpan ke '{output_path}'")

fig.show()

Chart berhasil disimpan ke 'bubble_cloud_chart_top5.png'
